# Podobnostní metriky

Tento notebook prezentuje různé metriky pro měření podobnosti mezi daty, včetně textových a numerických přístupů.


## 1. Levenshteinova vzdálenost

Levenshteinova vzdálenost měří minimální počet operací (vložení, odstranění, náhrada), nutných k transformaci jednoho řetězce na druhý.


In [ ]:
from textdistance import levenshtein

str1 = "kitten"
str2 = "sitting"

lev_dist = levenshtein(str1, str2)
print(f'Levenshteinova vzdálenost mezi "{str1}" a "{str2}" je {lev_dist}')

Levenshteinova vzdálenost mezi "kitten" a "sitting" je 3


## 2. Jaro-Winklerova podobnost

Jaro-Winklerova metrika je užitečná zejména při porovnávání jmen a adres, protože přikládá vyšší váhu shodám na začátku řetězce.


In [ ]:
from textdistance import jaro_winkler

str1 = "Robert"
str2 = "Rupert"

similarity = jaro_winkler(str1, str2)
print(f'Jaro-Winklerova podobnost mezi "{str1}" a "{str2}" je {similarity:.4f}')

Jaro-Winklerova podobnost mezi "Robert" a "Rupert" je 0.8000


## 3. Kosinová podobnost

Měří úhel mezi dvěma vektory reprezentujícími textové dokumenty nebo atributy.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = [
    "Toto je první dokument.",
    "Tento dokument je podobný prvnímu.",
    "Toto je zcela jiný text.",
]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)
cos_sim = cosine_similarity(tfidf_matrix)
print("Matice kosinové podobnosti:")
print(cos_sim)

Matice kosinové podobnosti:
[[1.         0.29558668 0.29558668]
 [0.29558668 1.         0.08882283]
 [0.29558668 0.08882283 1.        ]]


## 4. Jaccardův koeficient

Měří podobnost množin na základě jejich průniku a sjednocení.


In [ ]:
from sklearn.metrics import jaccard_score

set1 = {1, 2, 3, 4, 5}
set2 = {3, 4, 5, 6, 7}

jaccard_index = len(set1.intersection(set2)) / len(set1.union(set2))
print(f"Jaccardův koeficient je {jaccard_index:.4f}")

Jaccardův koeficient je 0.4286


## 5. TF-IDF

TF-IDF váží slova podle jejich četnosti v dokumentech a globální významnosti.


In [ ]:
import pandas as pd

df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print("TF-IDF reprezentace dokumentů:")
display(df)

TF-IDF reprezentace dokumentů:


,dokument,je,jiný,podobný,první,prvnímu,tento,text,toto,zcela
0,0.480458,0.373119,0.000000,0.000000,0.631745,0.000000,0.000000,0.000000,0.480458,0.000000
1,0.383770,0.298032,0.000000,0.504611,0.000000,0.504611,0.504611,0.000000,0.000000,0.000000
2,0.000000,0.298032,0.504611,0.000000,0.000000,0.000000,0.000000,0.504611,0.383770,0.504611


## 6. Numerické metriky

Euklidovská, Manhattanská a Mahalanobisova vzdálenost se využívají pro porovnávání číselných hodnot.


In [ ]:
from scipy.spatial.distance import euclidean, cityblock, mahalanobis
import numpy as np

point1 = np.array([2, 3, 5])
point2 = np.array([7, 1, 9])

euc_dist = euclidean(point1, point2)
manh_dist = cityblock(point1, point2)

print(f"Euklidovská vzdálenost: {euc_dist:.4f}")
print(f"Manhattanská vzdálenost: {manh_dist:.4f}")

Euklidovská vzdálenost: 6.7082
Manhattanská vzdálenost: 11.0000


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import numpy as np

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text.lower())  # Process text and convert to lowercase
    lemmatized_words = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]
    return " ".join(lemmatized_words)

# List of documents, where some may be duplicates
documents = [
    "Cheap red apples",
    "Cheap apples",
    "Red apples at a low price",
    "Best deal on red apples",
    "Green apples are healthy",
    "Quality green apples at a great price",
]

# Apply lemmatization to each document
lemmatized_documents = [lemmatize_text(doc) for doc in documents]
print(lemmatized_documents)

# Convert texts to TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(lemmatized_documents)

# Compute cosine similarity matrix
cos_sim_matrix = cosine_similarity(tfidf_matrix)

# Define threshold for duplicate detection
threshold = 0.2

# Find similar documents
duplicates = set()
for i in range(len(documents)):
    for j in range(i + 1, len(documents)):
        if cos_sim_matrix[i, j] > threshold:
            duplicates.add((documents[i], documents[j], cos_sim_matrix[i, j]))

# Print detected duplicates
for doc1, doc2, score in duplicates:
    print(f"Possible duplicates:\n - {doc1}\n - {doc2}\n Similarity score: {score:.2f}\n")


['cheap red apple', 'cheap apple', 'red apple low price', 'good deal red apple', 'green apple healthy', 'quality green apple great price']
Possible duplicates:
 - Red apples at a low price
 - Quality green apples at a great price
 Similarity score: 0.30

Possible duplicates:
 - Cheap red apples
 - Best deal on red apples
 Similarity score: 0.36

Possible duplicates:
 - Cheap red apples
 - Cheap apples
 Similarity score: 0.80

Possible duplicates:
 - Red apples at a low price
 - Best deal on red apples
 Similarity score: 0.27

Possible duplicates:
 - Green apples are healthy
 - Quality green apples at a great price
 Similarity score: 0.34

Possible duplicates:
 - Cheap red apples
 - Red apples at a low price
 Similarity score: 0.38

